In [1]:
import pandas as pd
import numpy as np

from fairlearn.metrics import MetricFrame
from fairlearn.metrics import count, \
                              false_positive_rate, \
                              selection_rate, equalized_odds_ratio
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds

 
                              
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from fairlearn.metrics import (
    demographic_parity_difference,
    equalized_odds_difference,
)
from fairlearn.reductions import (  # noqa
    DemographicParity,
    EqualizedOdds,
    ExponentiatedGradient,
)

from constants import protected_attributes

# Settings
np.random.seed(0)
import warnings
warnings.filterwarnings("ignore")

In [2]:

ds_train = pd.read_csv('./../data/train.csv')
ds_test = pd.read_csv('./../data/test.csv')


In [8]:
# Calculate correlation matrix
corr_matrix = ds_train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than a threshold
to_drop = [column for column in upper.columns if any(upper[column] > 0.995)]
# Drop features 
ds_train.drop(to_drop, axis=1, inplace=True)

[]


In [7]:
ds_train

,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,...,typering_hist_aantal,typering_hist_inburgeringsbehoeftig,typering_hist_ind,typering_hist_sector_zorg,typering_ind,typering_indicatie_geheime_gegevens,typering_other,typering_transport__logistiek___tuinbouw,typering_zorg__schoonmaak___welzijn,checked
0,4,2,0,1,9112,1,0,0,1,0,...,1,0,1,0,1,0,0,0,0,0
1,2,1,0,0,6016,1,0,0,0,0,...,2,0,1,0,1,0,0,0,0,0
2,3,1,0,1,22477,1,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
3,4,3,1,1,3520,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0
4,2,2,0,1,12511,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10111,4,3,0,1,913,1,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0
10112,2,1,1,0,10632,1,0,0,1,0,...,4,0,1,0,1,0,0,0,0,0
10113,3,3,2,0,7383,1,0,0,1,0,...,1,0,1,0,1,0,1,1,0,0
10114,5,2,0,0,10569,1,0,0,0,0,...,1,0,1,0,1,0,1,0,0,0


In [ ]:
# Let's specify the features and the target
y_train = ds_train["checked"]
X_train = ds_train.drop(['checked'], axis=1)
X_train = X_train.astype(np.float32)

# Let's specify the features and the target
y_test = ds_test["checked"]
X_test = ds_test.drop(['checked'], axis=1)
X_test = X_test.astype(np.float32)